In [71]:
import pandas as pd
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import re
from autocorrect import spell

In [72]:
import json
docs=[]
with open('data.json') as f:
    data = json.load(f)
count=0
for each in data:
    count+=1
    docs.append(each['title'])


vectorizer = TfidfVectorizer(stop_words='english',strip_accents='unicode',encoding='utf-8', decode_error='ignore')
temp = vectorizer.fit(docs)
temp2 = vectorizer.transform(docs)

In [4]:
print(temp2)

  (0, 3385)	0.05369452820101016
  (0, 3112)	0.23534711612929818
  (0, 2605)	0.2913921424224474
  (0, 2270)	0.21123395680454554
  (0, 2214)	0.2913921424224474
  (0, 1972)	0.2503999200695356
  (0, 1916)	0.24224337593365833
  (0, 1741)	0.11098649653956141
  (0, 1291)	0.06033441403613629
  (0, 1245)	0.2503999200695356
  (0, 1208)	0.2732527950627711
  (0, 1173)	0.2603827232933346
  (0, 702)	0.2913921424224474
  (0, 554)	0.2732527950627711
  (0, 375)	0.23534711612929818
  (0, 333)	0.2913921424224474
  (0, 153)	0.2913921424224474
  (1, 3481)	0.3768285506425073
  (1, 3111)	0.4018435700628122
  (1, 3028)	0.4018435700628122
  (1, 1916)	0.3340650924901203
  (1, 1291)	0.1664080309947379
  (1, 1132)	0.3590801119104251
  (1, 375)	0.3245548234868911
  (1, 272)	0.4018435700628122
  :	:
  (493, 841)	0.1808725622155888
  (493, 608)	0.38883284331073736
  (493, 367)	0.08860881528227592
  (493, 80)	0.13320221293403042
  (493, 71)	0.2040259428563695
  (493, 40)	0.18696269793548023
  (493, 38)	0.180872562215

In [5]:
import pickle

In [6]:
pickle.dump(temp, open("vectorizer.pickle", "wb"))
pickle.dump(temp2, open("tfidf.pickle", "wb"))

In [7]:
myvectorizer = pickle.load(open("vectorizer.pickle", "rb"))
tfidf = pickle.load(open("tfidf.pickle", "rb"))

In [161]:
from operator import itemgetter
import operator
query_string = ["minster ofice"]
query = 'boeing'
re_item = query 

#tfidf
query_vec = myvectorizer.transform(query_str)
cosine_similarities = linear_kernel(query, tfidf).flatten()
related_docs_indices = cosine_similarities.argsort()[:-20:-1]
res = [i for i in related_docs_indices if (cosine_similarities[i] > 0)]

#regular expression
for i,each in enumerate(data): 
    if re.search(re_item, each['name'], flags=re.IGNORECASE):
            res.append(i)
    for word in each['name'].split(' '):
        for look in query.split(' '):
            if re.search(look, word, flags=re.IGNORECASE):
                res.append(i)
                
for i,each in enumerate(data): 
    if re.search(re_item, each['title'], flags=re.IGNORECASE):
            res.append(i)
    for word in each['title'].split(' '):
        for look in query.split(' '):
            if re.search(look, word, flags=re.IGNORECASE):
                res.append(i)                
res = list(dict.fromkeys(filtered_docs_indices))

if len(filtered_docs_indices) == 0:
    query_string = [spell(query)]
    re_item = spell(query)
    query_vec = myvectorizer.transform(query_string)
    cosine_similarities = linear_kernel(query_vec, tfidf).flatten()
    related_docs_indices = cosine_similarities.argsort()[:-21:-1]
    filtered_docs_indices = [i for i in related_docs_indices if (cosine_similarities[i] > 0)]
    for i,each in enumerate(data): 
        if re.search(re_item, each['name'], flags=re.IGNORECASE):
                filtered_docs_indices.append(i)
        for word in each['name'].split(' '):
            for look in query.split(' '):
                if re.search(look, word, flags=re.IGNORECASE):
                    filtered_docs_indices.append(i)
    for i,each in enumerate(data): 
        if re.search(re_item, each['title'], flags=re.IGNORECASE):
                filtered_docs_indices.append(i)
        for word in each['title'].split(' '):
            for look in query.split(' '):
                if re.search(look, word, flags=re.IGNORECASE):
                    filtered_docs_indices.append(i) 

#consider follower and time
list_time_follower = []
list_time = []
list_follower = []
for i in filtered_docs_indices:
    list_time.append(data[i]['url'][2])
for i in filtered_docs_indices:
    list_follower.append(data[i]['followers_count'])

normed_time = [i/sum(list_time) for i in list_time]
normed_time = [x * 2 for x in normed_time]
normed_followers = [i/sum(list_follower) for i in list_follower]


lst= list(map(operator.add, normed_time,normed_followers))

order=[]
for idx, i in enumerate(filtered_docs_indices):
    order.append([i,lst[idx]])

order= sorted(order, key=itemgetter(1),reverse=True)
order2 = [item[0] for item in order]
print(order2)

    

[5, 32, 33, 36, 25, 26, 11, 19, 20, 23, 53, 74, 82, 143, 164, 166, 167, 168, 169, 170, 173, 163, 159, 160, 162, 247, 288, 305, 424, 430, 434, 435, 439, 440]


In [43]:
for i in filtered_docs_indices:
    print(data[i]['title'])